In [3]:
import subprocess
import logging
import os
import numpy as np
import random
import shutil
import platform
import sys
from typing import Union, List, Optional

In [4]:
class DependencyError(Exception):
    pass

In [16]:
class File(object):
    '''Creates File object that encapsulates a number of properites.
    
    Attributes (class and instance attributes):
        file (class and instance): Class variable that is set once class is instantiated.
    '''
    
    file: str = ""
    
    def __init__(self,
                 file: str) -> str:
        '''Init doc-string for File object class.
        
        Usage example:
            file_obj = File("<file_name.txt>")
            file_obj.file -> "<file_name.txt>" # returns filename str
        
        Args:
            file: Input file (need not exist at runtime/instantiated).
        '''
        self.file: str = file
        
    def touch(self) -> str:
        '''Creates empty file.
        
        Usage example:
            file_obj = File("<file_name.txt>")
            file_obj.touch()   # creates empty file
        '''
        with open(self.file,'w') as tmp_file:
            pass
        return self.file
    
    def abs_path(self) -> str:
        '''Returns absolute path of file.
        
        Usage example:
            file_obj = File("<file_name.txt>")
            file_obj.abs_path()   # returns abs path of file as str
        '''
        if os.path.exists(self.file):
            return os.path.abspath(self.file)
        else:
            self.touch()
            file_path = os.path.abspath(self.file)
            os.remove(self.file)
            return file_path
    
    def rm_ext(self,
              ext: str = "") -> str:
        '''Removes file extension from the file.
        
        Usage example:
            # Recommended
            file_obj = File("<file_name.txt>")
            file_obj.rm_ext("<.txt>")
            
            # or
            
            # Not recommended
            file_obj = File("<file_name.txt>")
            file_obj.rm_ext()   # returns "file_name"
        
        Args:
            ext: File extension.
        
        Returns:
            file: File name with no extension.
        '''
        self.ext: str = ext
        if self.ext is None:
            return self.file[:-(4)]
        else:
            ext_num = len(self.ext)
            return self.file[:-(ext_num)]
        
    def write_txt(self,
                 txt: str = "") -> str:
        '''Writes/appends text to file.
        
        Usage example:
            file_obj = File("<file_name.txt>")
            file_obj.write_txt("<Some text to be written>")
        
        Args:
            txt: Text to be written to file.
        Returns:
            file: File with text written/appended to it.
        '''
        with open(self.file,"a") as tmp_file:
            tmp_file.write(txt)
            tmp_file.close()
        return self.file

In [17]:
f = File("test.txt")

In [15]:
f.

'test.txt'

In [18]:
os.remove(f.file)

In [5]:
class TmpDir(object):
    '''Temporary directory class that creates temporary directories and files
    given a parent directory.
    
    Attributes (class and instance attributes):
        tmp_dir (class and instance): Temproary directory.
        parent_tmp_dir (class and instance): Input parent directory.
    '''
    
    # Set parent tmp directory, as tmp_dir is overwritten
    tmp_dir = ""
    parent_tmp_dir = ""
    
    def __init__(self,
                tmp_dir: str,
                use_cwd: bool = False):
        '''Init doc-string for TmpDir class.
        
        Args:
            tmp_dir: Temporary parent directory name/path.
            use_cwd: Use current working directory as working direcory.
        '''
        _n = 10000 # maximum N for random number generator
        tmp_dir = os.path.join(tmp_dir,'tmp_dir_' + 
                               str(random.randint(0,_n)))
        self.tmp_dir = tmp_dir
        self.parent_tmp_dir = os.path.dirname(self.tmp_dir)
        if use_cwd:
            _cwd = os.getcwd()
            self.tmp_dir = os.path.join(_cwd,self.tmp_dir)
            self.parent_tmp_dir = os.path.dirname(self.tmp_dir)
        
    def mk_tmp_dir(self) -> None:
        '''Creates/makes temporary directory.'''
        if not os.path.exists(self.tmp_dir):
            return os.makedirs(self.tmp_dir)
        else:
            print("Temporary directory already exists")
        
    def rm_tmp_dir(self,
                  rm_parent: bool = False) -> None:
        '''Removes temporary directory.
        
        Args:
            rm_parent: Removes parent directory as well.
        '''
        if rm_parent and os.path.exists(self.parent_tmp_dir):
            return shutil.rmtree(self.parent_tmp_dir)
        elif os.path.exists(self.tmp_dir):
            return shutil.rmtree(self.tmp_dir)
        else:
            print("Temporary directory does not exist")
    
    class TmpFile(File):
        '''Child/sub-class of TmpDir. Creates temporary files by inheriting File object
        properties from the File class.
        
        Attributes (class and instance attributes):
            tmp_file (class and instance): Temporary file name.
        '''
        
        tmp_file = ""
        tmp_dir = ""

        def __init__(self,
                    tmp_file: File,
                    tmp_dir: str = ""):
            '''Init doc-string for TmpFile class. Allows for creation of 
            a temporary file in parent class' temporary directory location.
            
            Args:
                tmp_file: Temporary file name.
                tmp_dir: Temporary directory name.
            '''
            self.tmp_file = tmp_file
            self.tmp_dir = tmp_dir
            self.tmp_file = os.path.join(self.tmp_dir,self.tmp_file)
            File.__init__(self,self.tmp_file)

In [6]:
d = "test.dir"

In [7]:
dd = TmpDir(d, True)

In [8]:
dd.mk_tmp_dir()

In [9]:
f = 'test.file'

In [10]:
dd.tmp_dir

'/mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/test.dir/tmp_dir_2120'

In [11]:
ff = TmpDir.TmpFile(f,dd.tmp_dir)

In [12]:
ff.touch()

'/mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/test.dir/tmp_dir_2120/test.file'

In [13]:
ff.tmp_dir

'/mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/test.dir/tmp_dir_2120'

In [14]:
ff.abs_path()

'/mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/test.dir/tmp_dir_2120/test.file'

In [15]:
ff.tmp_file

'/mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/test.dir/tmp_dir_2120/test.file'

In [16]:
ff.tmp_file

'/mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/test.dir/tmp_dir_2120/test.file'

In [17]:
ff.touch()

'/mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/test.dir/tmp_dir_2120/test.file'

In [18]:
dd.rm_tmp_dir(True)

In [20]:
# os.remove("test.file")

In [21]:
class NiiFile(File):
    '''NIFTI file class object for handling NIFTI files. Inherits properties
    from the File class.
    
    Attributes (class and instance attributes):
        file (class and instance): NIFTI file path.
    '''
    def __init__(self,
                 file: File):
        '''Init doc-string for NiiFile class.'''
        self.file = file
        File.__init__(self,self.file)
    
    def rm_ext(self):
        '''
        Intended for use-cases pertaining to NIFTI files.
        Removes file extensions of NIFTI files.
        '''
        if '.nii.gz' in self.file:
            return self.file[:-7]
        elif '.nii' in self.file:
            return self.file[:-4]
        else:
            return self.file

In [22]:
class LogFile(File):
    '''Class that creates a log file for logging purposes. Due to how this 
    class is constructed - its intended use cases requires that this class 
    is instantiated/called once and ONLY once.
    
    Once a class instance has been instantiated, then it and its associated
    methods can be used.
    
    Attributes (class and instance attributes):
        log_file: Log file filename.
    '''
    
    # # Define global variables
    # global log_file
    # global logging
    # global logger
    # global console
    
    def __init__(self,
                 log_file: File = "") -> None:
        '''Init doc-string for LogFile class.
        
        Args:
            file: Log filename (need not exist at runtime).
        '''
        self.log_file: File = log_file
        
        # Set-up logging to file
        logging.basicConfig(level=logging.INFO,
                            format='%(asctime)s %(name)-12s %(levelname)-8s %(message)s',
                            datefmt='%d-%m-%y %H:%M:%S',
                            filename=self.log_file,
                            filemode='a')
        
        # Define a Handler which writes INFO messages or higher to the sys.stderr
        self.console = logging.StreamHandler()
        self.console.setLevel(logging.INFO)
        
        # Add the handler to the root logger
        logging.getLogger().addHandler(self.console)
        
        # Define logging
        self.logger = logging.getLogger(__name__)
        
    def info(self,
            msg: str = "") -> None:
        '''Writes information to log file.'''
        self.logger.info(msg)
        
    def debug(self,
            msg: str = "") -> None:
        '''Writes debug information to file.'''
        self.logger.debug(msg)
        
    def error(self,
            msg: str = "") -> None:
        '''Writes error information to file.'''
        self.logger.error(msg)
    
    def log(self,
        log_cmd: str = ""):
        '''
        Log function for logging commands and messages to some log file.

        Usage:
            # Initialize the `log` function command
            log_msg = Command("log")

            # Specify output file and message
            log_msg.log("sub.log","test message 1")

            # Record message, however - no need to re-initialize `log` funcion command or log output file
            log_msg.log("test message 2")

        NOTE: The input `log_file` only needs to be specified once. Once specified,
            this log is written to each time this or the `run` function is invoked.

        Arguments:
            log_file(file): Log file to be written to. 
            log_cmd(str): Message to be written to log file
        '''

        # Log command/message
        self.info(log_cmd)

In [23]:
# Needs to be instantiated once and ONLY once
# log = LogFile("sub-test.2.log")
log = LogFile("")

In [24]:
class Command(object):
    '''
    Creates a command and an empty command list for UNIX command line programs/applications. Primary use and
    use-cases are intended for the subprocess module and its associated classes (i.e. Popen/call/run).
    
    Attributes (class and instance attributes):
        command (instance): Command to be performed on the command line.
        cmd_list (instance): Mutable list that can be appended to.
    
    Modules/Packages required:
        - os
        - logging
        - subprocess
    '''

    def __init__(self,
                 command: str) -> list:
        '''
        Init doc-string for Command class. Initializes a command to be used on UNIX command line.
        The input argument is a command (string), and a mutable list is returned (, that can later
        be appended to).
        
        Usage:
            echo = Command("echo")
            echo.cmd_list.append("Hi!")
            echo.cmd_list.append("I have arrived!")
        
        Arguments:
            command (string): Command to be used. Note: command used must be in system path
        Returns:
            cmd_list (list): Mutable list that can be appended to.
        '''
        self.command = command
        self.cmd_list = [f"{self.command}"]
        
    def check_dependency(self,
                 err_msg: Optional[str] = None) -> None:
        '''Checks dependency of some command line executable/command.
        Should the dependency not be met, then an exception is raised.
        Check system path should problems arise and ensure that the executable
        of interest is installed.
        
        Args:
            err_msg: Error message to print to screen.
        
        Raises:
            DependencyError: Dependency error exception is raised if the dependency
                is not met.
        '''
        if not shutil.which(self.command):
            if err_msg:
                print(f"\n \t {err_msg} \n")
            else:
                print(f"\n \t The required dependency {self.command} is not installed or in the system path. \n")
            raise DependencyError("Command executable not found in system path.")
        
    def run(self,
            debug: bool = False,
            dryrun: bool = False,
            env: dict = {},
            stdout: File = "",
            shell: bool = False
           ) -> List[Union[int,str]]:
        '''
        Uses python's built-in subprocess class to execute (run) a command from an input command list.
        The standard output and error can optionally be written to file.
        
        Usage:
            echo.run() # This will return tuple (returncode,log,None,None), but will echo "Hi!" to screen.
            
        NOTE: 
            - REQUIRED: `LogFile` `class` must be instantiated/called globally with the class variable instance
                named `log` for this class to run properly.
            - The contents of the 'stdout' output file will be empty if 'shell' is set to True.
            - Once the log file name 'log_file' has been set, that value is stored and cannot be changed.
                - This log file will continue to be appended to for each invocation of this class.
        Arguments:
            log_file(file): Output log file name.
            debug(bool): Sets logging function verbosity to DEBUG level
            dryrun(bool): Dry run -- does not run task. Command is recorded to log file.
            env(dict): Dictionary of environment variables to add to subshell.
            stdout(file): Output file to write standard output to.
            shell(bool): Use shell to execute command.
        Returns:
            p.returncode(int): Return code for command execution should the 'log_file' option be used.
            log_file(file): Output log file with appended information should the 'log_file' option be used.
            stdout(file): Standard output writtent to file should the 'stdout' option be used.
            stderr(file): Standard error writtent to file should the 'stdout' option be used.
        '''
        
        # Create command str for log
        cmd: str = ' '.join(self.cmd_list) # Join list for logging purposes
        
        if debug:
            log.debug(f"Running: {cmd}")
        else:
            log.info(f"Running: {cmd}")
        
        if dryrun:
            log.info("Performing command as dryrun")
            return 0
        
        # Define environment variables
        merged_env: dict = os.environ
        if env:
            merged_env.update(env)
        
        # Execute/run command
        p = subprocess.Popen(self.cmd_list,shell=shell,env=merged_env,
                        stdout=subprocess.PIPE,stderr=subprocess.PIPE)

        # Write log files
        out,err = p.communicate()
        out = out.decode('utf-8')
        err = err.decode('utf-8')

        # Write std output/error files
        if stdout:
            stderr: str = os.path.splitext(stdout)[0] + ".err"
                
            stdout: File = File(stdout)
            stderr: File = File(stderr)
            
            stdout.write_txt(out)
            stderr.write_txt(err)
        else:
            stdout = None
            stderr = None

        if p.returncode:
            log.error(f"command: {cmd} \n Failed with returncode {p.returncode}")

        if len(out) > 0:
            if debug:
                log.debug(out)
            else:
                log.info(out)

        if len(err) > 0:
            if debug:
                log.info(err)
            else:
                log.warning(err)
        return p.returncode,stdout,stderr

str

In [9]:
fsl_math = Command("fslmaths")

In [10]:
fsl_math.cmd_list.append("-h")

In [11]:
fsl_math.cmd_list

['fslmaths', '-h']

In [12]:
fsl_math.run()

Running: fslmaths -h

Usage: fslmaths [-dt <datatype>] <first_input> [operations and inputs] <output> [-odt <datatype>]

Datatype information:
 -dt sets the datatype used internally for calculations (default float for all except double images)
 -odt sets the output datatype ( default is float )
 Possible datatypes are: char short int float double input
 "input" will set the datatype to that of the original image

Binary operations:
  (some inputs can be either an image or a number)
 -add   : add following input to current image
 -sub   : subtract following input from current image
 -mul   : multiply current image by following input
 -div   : divide current image by following input
 -rem   : modulus remainder - divide current image by following input and take remainder
 -mas   : use (following image>0) to mask current image
 -thr   : use following number to threshold current image (zero anything below the number)
 -thrp  : use following percentage (0-100) of ROBUST RANGE to threshold cu

(0, None, None)

In [120]:
os.remove("sub-test.2.log")

In [39]:
fsl_math.check_dependency()

In [13]:
fsl_val = Command("fslval")

In [14]:
fsl_val.run()

Running: fslval
command: fslval 
 Failed with returncode 1
Usage: fslval <input> <keyword>



(1, None, None)